# **Ejecución Modelo Personalizado CNN en BirdNET Analyzer**
versión 2.1 (20-02-2025)

### *Ron A. Fernández-Gómez - EPM SECIHTI*
(puntuacion omitida)
Este es un flujo de trabajo en Google Colab escrito en Python para ejecutar modelos personalizados basados en Redes Neuronales Convolucionales (CNNs) para detección y clasificación de vocalizaciones de fauna utilizando la herramienta BirdNET Analyzer para uso con Interfaz de línea de comando (CLI).
Para mayores detalles sobre BirdNET Analyzer y usos del proyecto BirdNET consultar: https://github.com/kahst/BirdNET-Analyzer


Aunque los modelos compatibles con BirdNET Analyzer se generan en Tensorflow, algunos están en formato .tflite que pueden tener algunas limitaciones para optimizar procesos de inferencias basados en el uso de paralelización en GPU, delegando los procesos a la CPU. Por lo tanto en este flujo buscaremos trabajar en un entorno de ejecución de colab con recursos disponibles en CPU. Se recomienda usar v2-8 TPU que cuenta con la virtualización de 96 procesadores CPU.


# 1. INFERENCIA: Deteccion de vocalizaciones por lotes
Esta etapa corresponde a la ejecución de un modelo CNN por lotes. Permite ejecutar el modelo en una carpeta que contenga varios archivos de audio.

## Paqueterías y dependencias necesarias
Estas dependencias facilitaran el funcionamiento del analizador

In [ ]:
!pip install ffmpeg
!pip install librosa
!pip install resampy
!pip install tensorflow==2.15.0
!pip install keras-tuner

##*Conectar a Google Drive y definir rutas*
Primero vincularemos Google Drive con lo que definirimos las rutas de acceso a los archivos de trabajo y de salida de resultados durante la sesión de trabajo. En el Drive tedremos los archivos a analizar y los modelos personalizados a ejecutar.

In [ ]:
# Conectar con google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Para trabajar con varios archivos hay que definir
#Rutas de entrada y salida
audio_path = '/ruta/a/la/carpeta/de/archivos' #definir esta ruta
output_folder = '/ruta/a/la/carpeta/donde/guardar/los/resultados' #definir esta ruta

#Ruta al modelo personalizado
my_model = '/ruta/al/modelo/personalizado.tflite' #definir esta ruta

## Clonar BirdNET Analyzer
Clonamos el respositorio de BirdNET Analyzer en nuestro espacio de trabajo a partir de GitHub

In [ ]:
!git clone https://github.com/kahst/BirdNET-Analyzer

In [ ]:
#Para trabajar con varios archivos hay que definir
#Rutas de entrada y salida
audio_path = '/content/drive/Shareddrives/SOUNDSCAPES SELVA MAYA P2/2024.10-11_Oct-Nov/QR-FCP__Dzula__R202---FLAC/Data' #definir esta ruta
output_folder = '/content/drive/MyDrive/BirdNet_results/outputdzulaR202' #definir esta ruta

In [ ]:
%cd BirdNET-Analyzer
#%cd /content

/content/BirdNET-Analyzer


In [ ]:
!pwd

/content/BirdNET-Analyzer


In [ ]:
#Hacemos una prueba rapida con example para verificar funcionamiento correcto
!python -m birdnet_analyzer.analyze \
        --combine_results

2025-02-03 04:46:42.596125: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738558002.654282    5488 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738558002.672810    5488 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-03 04:46:42.740232: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Species list contains 6522 species
Found 1 files to analyze
Analyzing /content/BirdNET-Analyzer/birdnet_analyzer/example/soun

In [ ]:
# Procesar una carpeta con archivos y el modelo personalizado, en este caso cargamos el modelo de WildMon dando la ruta del archivo
!python -m birdnet_analyzer.analyze \
        --i "{audio_path}" \
        --o "{output_folder}" \
        --classifier "{my_model}" \
        --min_conf 0.5 \ #definimos el minimo de confianza
        --threads 96 \ #denifimos la cantidad de hilos de procesadores CPU a utilizar, por ejmplo 96 el maximo del entorno de ejecución
        --batchsize 32 \ #definimos la cantidad de archivos a anlizar al tiempo.
        --combine_results #solo se declara si se quiere unificar resultados en un archivo



Con esto obtendremos un listado cvs con todas las detecciones